In [1]:
import pandas as pd
import numpy as np
import json
import requests
from datetime import datetime

In [16]:
req = requests.get('https://min-api.cryptocompare.com/data/price?fsym=BTC&tsyms=USD,JPY,EUR')

In [17]:
req.status_code

200

In [19]:
req.text

'{"USD":10412.45,"JPY":1112032.14,"EUR":9420.17}'

In [22]:
req_json = req.json()

In [29]:
req_json

{'USD': 10412.45, 'JPY': 1112032.14, 'EUR': 9420.17}

**Using Parameters**

In [2]:
def return_json(url, params):
    req = requests.get(url, params)
#     print("Status code:", req.status_code)
    req_json = req.json()
    return req_json

def create_df(original, new_json):
    new = pd.DataFrame(new_json['Data'])
    new.sort_values('time', ascending= False, inplace = True)
    new = pd.concat([original,new], axis = 0)
    return new

In [3]:
url = 'https://min-api.cryptocompare.com/data/histohour'
# url = "https://min-api.cryptocompare.com/data/v2/histominute"

In [15]:
df_btc = pd.DataFrame()
df_eth = pd.DataFrame()
df_eos = pd.DataFrame()
df_ltc = pd.DataFrame()
df_xrp = pd.DataFrame()
df_dash = pd.DataFrame()

In [16]:
list_of_df = [df_btc, df_eth, df_eos, df_ltc, df_xrp, df_dash]
list_of_curr = ['BTC', 'ETH', 'EOS', 'LTC', 'XRP','DASH']
for j in range(len(list_of_curr)):
    start = datetime.now()
    ts = 1567814400
    for i in range(50):
        params = {'fsym': [list_of_curr[j]], 'tsym':['USD'], 'aggregate':[1],'limit': 2000, 'toTs':ts}
        req_json = return_json(url, params)
        
#         if i==0:
#             list_of_df[j] = pd.DataFrame(req_json['Data'])
#             list_of_df[j].sort_values('time', ascending= False, inplace = True)
#         else:

        list_of_df[j] = create_df(list_of_df[j], req_json)

        ts = list_of_df[j].time.iloc[-1]
    list_of_df[j]['dt'] = list_of_df[j].time.apply(lambda x : pd.Timestamp(x, unit='s', tz = 'GMT'))    
    list_of_df[j] = list_of_df[j].drop_duplicates(subset = 'dt')
    list_of_df[j]['year'] = list_of_df[j].dt.apply(lambda x: x.year)
#     list_of_df[j] = list_of_df[j][list_of_df[j].year > 2014]  
    list_of_df[j] = list_of_df[j].reset_index().drop('index', axis =1)
    list_of_df[j] = list_of_df[j].set_index('dt')
    
    print("time taken : ", datetime.now()-start)
#     list_of_df[j] = list_of_df[j].drop_duplicates(subset = ['close', 'high', 'low', 'open', 'volumefrom', 'volumeto'], keep='first')
# #     if (list_of_df[j].dt.shift() - list_of_df[j].dt).nunique() != 1:
# #         print("Defaulter :", j)

time taken :  0:00:55.229907
time taken :  0:00:49.660096
time taken :  0:00:48.109085
time taken :  0:00:52.575809
time taken :  0:00:50.356469
time taken :  0:00:49.067946


In [17]:
df_btc = list_of_df[0]
df_eth = list_of_df[1]
df_eos = list_of_df[2]
df_ltc = list_of_df[3]
df_xrp = list_of_df[4]
df_dash = list_of_df[5]

In [71]:
# ethereum launch date onwards
df_eth = df_eth.loc[:'2015-08-01',:]
# btc launch date onwards
df_btc = df_btc.loc[:'2010-7', :]
# eos launch date onwards
df_eos = df_eos.loc[:'2018-01-31', :]
# ltc launch date onwards
df_ltc = df_ltc.loc[:'2011-10-07', :]
# dash launch date onwards
df_dash = df_dash.loc[:'2014-01-18', :]
# xrp launch date onwards
df_xrp = df_xrp.loc[:'2015-01-01', :]

In [57]:
df_eth.to_csv('../Desktop/eth.csv')
df_eos.to_csv('../Desktop/eos.csv')
df_ltc.to_csv('../Desktop/ltc.csv')
df_dash.to_csv('../Desktop/dash.csv')
df_xrp.to_csv('../Desktop/xrp.csv')
df_btc.to_csv('../Desktop/btc.csv')